<a href="https://colab.research.google.com/github/jerry26432341/myTheses/blob/main/0426_newdata_lnsvc_%E8%B6%85%E5%8F%83%E6%95%B885_60.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas matplotlib jieba scikit-learn gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import jieba as jb
import re
from google.colab import drive


# 掛載Google Drive
drive.mount('/content/drive')

# 讀取CSV檔案
df = pd.read_csv('/content/drive/MyDrive/toutiao_cat_data.csv')
df=df[['cat','review']]
print("数据总量: %d ." % len(df))
df.sample(10)

Mounted at /content/drive


<ipython-input-2-50e193adac36>:15: DtypeWarning: Columns (5,6,7,8,9,10,11,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/toutiao_cat_data.csv')


数据总量: 382688 .


,cat,review
225862,travel,茶花的花苞总是不开花该怎么办？
147243,culture,上联：鸦鹊枝头谈福祸，下联怎么对？
260965,story,跟丈夫吵架我回娘家，查出怀孕等丈夫来接，却等到丈夫相亲的消息
328832,finance,一位血亏老股民的肺腑独白：这就是炒股人的命运，无法改变！
135139,travel,把耳朵贴在铁轨上，为什么就能听到远处火车声？
67725,story,你的消费习惯，暴露了你贫穷的根源
123519,travel,英国推出太空游套餐159万起
274122,car,男儿壮志未酬怎能甘于人下，五菱宏光，早晚都是一条好汉
24798,entertainment,李毓芬，网友：女神好漂亮啊，好有气质，好甜心啊
133233,travel,2018年5月8日，游客去遵义市旧居，看到了这样的画面


In [ ]:
print("在 cat 列中总共有 %d 个空值." % df['cat'].isnull().sum())
print("在 review 列中总共有 %d 个空值." % df['review'].isnull().sum())
df[df.isnull().values==True]
df = df[pd.notnull(df['review'])]

在 cat 列中总共有 0 个空值.
在 review 列中总共有 0 个空值.


In [ ]:
df['cat_id'] = df['cat'].factorize()[0]
cat_id_df = df[['cat', 'cat_id']].drop_duplicates().sort_values('cat_id').reset_index(drop=True)
cat_to_id = dict(cat_id_df.values)
id_to_cat = dict(cat_id_df[['cat_id', 'cat']].values)
df.sample(10)

,cat,review,cat_id
51944,game,孙尚香的团战精神皮肤是款福利皮肤，那么获取方式是怎样的呢？,12
359303,world,投票事件过后，联想今后的日子该怎样走？,10
352272,world,印度乡村为何那么破烂不堪？,10
233801,entertainment,先苦后甜、先甜后苦：不同的选择折射出不同的人生态度和人生轨迹,1
188505,travel,惊喜！再上27天班，又要放假啦！还有好些知名景点免费玩,9
27728,edu,九中——我新的起点,6
254970,entertainment,关晓彤晒美照纪念与鹿晗相恋七个月，低调秀恩爱一个小细节太甜了,1
144914,game,《FGO》两仪式从者宝具技能详细资料一览,12
209659,finance,“海南数字自由贸易发展路径与策略高端研讨会”在三亚圆满召开,3
193113,entertainment,"《创造101》里的""村花""杨超越，大家喜欢她吗？",1


In [ ]:
#定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line

def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

#加载停用词
stopwords = stopwordslist('/content/drive/MyDrive/chineseStopWords.txt')


In [ ]:
#删除除字母,数字，汉字以外的所有符号
df['clean_review'] = df['review'].apply(remove_punctuation)
df.sample(10)

,cat,review,cat_id,clean_review
241130,finance,如何评价平安保险？,3,如何评价平安保险
102018,sports,巅峰不会有第二次？三大英超教练难以再重现昨日辉煌，穆帅居首,2,巅峰不会有第二次三大英超教练难以再重现昨日辉煌穆帅居首
52182,house,廉租房到期后，会出现什么情况？,4,廉租房到期后会出现什么情况
292047,game,斗地主：只要底牌有张4，我就可以超级加倍打天下了,12,斗地主只要底牌有张4我就可以超级加倍打天下了
317760,tech,怎么看待靠标题来吸引点击的自媒体？,7,怎么看待靠标题来吸引点击的自媒体
295228,tech,微信的聊天功能很厉害，聊天记录居然有提醒功能，还能克服打字慢,7,微信的聊天功能很厉害聊天记录居然有提醒功能还能克服打字慢
246847,culture,孤独的孩子丨Gehard Demetz木刻雕塑,0,孤独的孩子丨GehardDemetz木刻雕塑
16658,sports,多了新身份！奥运冠军吴敏霞当选上海团市委副书记,2,多了新身份奥运冠军吴敏霞当选上海团市委副书记
319705,house,养老定居，选择珠三角城市好还是长三角城市好？,4,养老定居选择珠三角城市好还是长三角城市好
50611,house,有去香港工作的机会，应不应该去？,4,有去香港工作的机会应不应该去


In [ ]:
#分词，并过滤停用词
df['cut_review'] = df['clean_review'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
df.head()

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.082 seconds.
DEBUG:jieba:Loading model cost 1.082 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


,cat,review,cat_id,clean_review,cut_review
0,culture,京城最值得你来场文化之旅的博物馆,0,京城最值得你来场文化之旅的博物馆,京城 值得 来场 文化 之旅 博物馆
1,culture,发酵床的垫料种类有哪些？哪种更好？,0,发酵床的垫料种类有哪些哪种更好,发酵 床 垫料 种类 种 更好
2,culture,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,0,上联黄山黄河黄皮肤黄土高原怎么对下联,上联 黄山 黄河 黄皮肤 黄土高原 下联
3,culture,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,0,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣,林徽因 理由 拒绝 徐志摩 选择 梁思成 终身伴侣
4,culture,黄杨木是什么树？,0,黄杨木是什么树,黄杨木 树


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
nltk.download('punkt')
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
import numpy as np

# TF-IDF特徵提取
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, max_features=20000)
tfidf_features = tfidf_vectorizer.fit_transform(df['cut_review'])
tfidf_features_sparse = csr_matrix(tfidf_features)

# Doc2Vec特征提取
def tagged_document_generator(df):
    for i, doc in enumerate(df['cut_review']):
        yield TaggedDocument(nltk.word_tokenize(doc), [i])

tagged_documents = [TaggedDocument(nltk.word_tokenize(doc), [i]) for i, doc in enumerate(df['cut_review'])]
doc2vec_model = Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=10, workers=4)

doc2vec_features = [doc2vec_model.infer_vector(nltk.word_tokenize(doc)) for doc in df['cut_review']]
doc2vec_array = np.array(doc2vec_features)

# 將tfidf特徵和doc2vec特徵合併
features_sparse = hstack([csr_matrix(doc2vec_array), tfidf_features_sparse])

# 劃分數據集
X_train, X_test, y_train, y_test = train_test_split(features_sparse, df['cat'], test_size=0.2, random_state=42)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 初始化线性支持向量机分类器
svm_model = LinearSVC(random_state=42)

# 设置待调优的超参数及其取值范围
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

# 使用GridSearchCV进行超参数调优
grid_search = GridSearchCV(svm_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# 获取最佳超参数组合
best_params = grid_search.best_params_

# 使用最佳超参数训练模型
svm_model_best = LinearSVC(random_state=42, **best_params)
svm_model_best.fit(X_train, y_train)

# 预测测试集
y_pred = svm_model_best.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print("准确率: {:.2f}%".format(accuracy * 100))





/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

准确率: 85.60%
